# 将棋固有の表現を形態素解析するため、SentencePieceを作るノートブック

### 参考ページ
[SentencePieceを使ってみた。](https://www.pytry3g.com/entry/how-to-use-sentencepiece)

In [ ]:
# 自分のマイドライブにマウントする

from google.colab import drive
drive.mount('/content/drive/')

# カレントディレクトリの変更
import os
os.chdir("/content/drive/My Drive/God's Move/model")

# カレントディレクトリの取得
print(os.getcwd())

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/God's Move/Colab Notebooks


In [ ]:
!pip install sentencepiece > /dev/null

In [ ]:
# corpusを読み込み（別ノートブックでpickle保存していた)
import pickle

def loadPickle(fileName):
    with open(fileName, mode="rb") as f:
        return pickle.load(f)

def dumpPickle(fileName, obj):
    with open(fileName, mode="wb") as f:
        pickle.dump(obj, f)

# corpusデータを読み込み
corpus = loadPickle("../corpus.pickle")
text = "".join(corpus)

print("len_corpus: {}".format(len(corpus)))
print("len_text: {}".format(len(text)))

# pickle形式で保存
dumpPickle("../text.pickle", text)


len_corpus: 595775
len_text: 24353736


In [ ]:
import sentencepiece as spm
import codecs

# SentencePiece Train用にコーパスリストをテキストファイルに書き込み
with codecs.open("kif.txt", "w", "utf-8") as f:
    for text in corpus:
        f.write(text+"\n")

spm.SentencePieceTrainer.Train(
    '--input=kif.txt, --model_prefix=shogi --character_coverage=0.9995 --vocab_size=8000 --pad_id=3'
)

## 作成したSentencePieceから将棋固有の表現がID化されているか確認

In [ ]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.Load("shogi.model")

True

### EncodeAsPieces (テキストを単語に分割)

In [ ]:
print(corpus[50])
#先手は金開きに構えた。バランスがよく、先手番でよく用いられる。
print(sp.EncodeAsPieces(corpus[50]))
#['▁先手は', '金', '開き', 'に構えた', '。', 'バランス', 'がよく', '、', '先手番', 'で', 'よく', '用', 'い', 'られる', '。']

先手は金開きに構えた。バランスがよく、先手番でよく用いられる。
['▁先手は', '金', '開き', 'に構えた', '。', 'バランス', 'がよく', '、', '先手番', 'で', 'よく', '用', 'い', 'られる', '。']


### EncodeAsIds (テキストを単語ID列に変換)

In [ ]:
print(corpus[50])
#先手は金開きに構えた。バランスがよく、先手番でよく用いられる。
print(sp.EncodeAsIds(corpus[50]))
#[196, 222, 4006, 1211, 4, 1172, 2143, 5, 1783, 15, 439, 1005, 95, 341, 4]

先手は金開きに構えた。バランスがよく、先手番でよく用いられる。
[196, 222, 4006, 1211, 4, 1172, 2143, 5, 1783, 15, 439, 1005, 95, 341, 4]


### DecodePieces (単語列からテキストに変換)



In [ ]:
tokens = ['先手', 'は', '居飛車', 'を', '採用', '。']
print(sp.DecodePieces(tokens))
#先手は居飛車を採用。

先手は居飛車を採用。


### DecodeIds (単語ID列からテキストに変換)

In [ ]:
ids = sp.EncodeAsIds(corpus[50])
print(sp.DecodeIds(ids))
#先手は金開きに構えた。バランスがよく、先手番でよく用いられる。

先手は金開きに構えた。バランスがよく、先手番でよく用いられる。


### GetPieceSize (語彙数)

In [ ]:
print(sp.GetPieceSize())
#8000
#len(sp)でも語彙数が取得できる。

8000


### PieceToId (単語IDの取得)

In [ ]:
print(sp.PieceToId('羽生'))
print(sp.PieceToId('森内'))
print(sp.PieceToId('居飛車'))
print(sp.PieceToId('振り飛車'))
print(sp.PieceToId('四間飛車'))
print(sp.PieceToId('王手飛車'))
print(sp.PieceToId('穴熊'))
print(sp.PieceToId('矢倉'))
print(sp.PieceToId('名人'))
print(sp.PieceToId('永世名人'))
print(sp.PieceToId('永世竜王'))
print(sp.PieceToId('羽生善治'))

#sp['</s>']でも単語IDが取得できる。

275
515
318
403
747
2831
307
209
1228
0
0
3546


In [ ]:
for i in sp.EncodeAsIds("藤井聡太が神の一手を差した"):
    print(i, sp.IdToPiece(i))

6 ▁
6395 藤井聡
1831 太
8 が
4743 神
700 の一手
16 を
1666 差
54 した


In [ ]:
for i in sp.EncodeAsIds("ヤンキースのイチローがレイザービームを放った"):
    print(i, sp.IdToPiece(i))

6 ▁
5856 ヤ
564 ン
2615 キ
272 ー
636 ス
10 の
1160 イ
1269 チ
2330 ロ
272 ー
8 が
2221 レ
1160 イ
7418 ザ
272 ー
2789 ビ
6163 ーム
2778 を放った


### 作成時間はおおよそ10分程度でできた。